# カイ二乗検定
原因系変数が質的変数で、結果系変数が質的変数の場合において分析方法の一つにクロス集計表がある。クロス集計表では個々の分析ではオッズ比やリスク比があるが、全体ではカイ二乗検定を使用する。

カイ二乗検定では帰無仮説が「理論値と実測値は同じ」となるためp値は「理論値と実測値がたまたま同じ確率」となる。

また、カイ二乗検定では調整済み標準化誤差を算出することで、クロス集計表のセルごとにp値を算出することができる。

## ライブラリのインポート

In [1]:
import pandas as pd
import scipy as sp
import scipy.stats as stats
import numpy as np

## データの読み込み

In [2]:
df = pd.read_csv("golf.csv",encoding="shift-jis")
df.head()

,天気,気温,湿度,風,ゴルフ
0,晴,29,85,弱,しない
1,晴,27,90,強,しない
2,曇,28,78,弱,する
3,雨,21,96,弱,する
4,雨,20,80,弱,する


## クロス集計表の作成
ここでは目的変数を「ゴルフ」とし、原因変数を「天気」と「風」にする。

In [3]:
cross = pd.crosstab(df["ゴルフ"],[df["天気"],df["風"]])
cross

天気   晴     曇     雨   
風    弱  強  弱  強  弱  強
ゴルフ                  
しない  2  1  0  0  0  2
する   1  1  2  2  3  0

## p値

In [4]:
x2, p, df, expected=sp.stats.chi2_contingency(cross)

In [5]:
print(p)

0.11235865025855099


p値からたまたま実測値と理論値が同じになる確率は約11%となり、仮に帰無仮説棄却域を5%とした場合は帰無仮説を棄却出来ないことが分かる。

## 理論値の確認

In [6]:
dfe = pd.DataFrame(expected)
dfe.columns = cross.columns
dfe.index = cross.index
dfe

天気          晴                   曇                   雨          
風           弱         強         弱         強         弱         強
ゴルフ                                                            
しない  1.071429  0.714286  0.714286  0.714286  1.071429  0.714286
する   1.928571  1.285714  1.285714  1.285714  1.928571  1.285714

## 自由度
基本的に自由度は5を超えるようにするのが一般的となっている

In [7]:
df

5

## 実測値と理論値の差

In [8]:
diff = cross-expected
diff

天気          晴                   曇                   雨          
風           弱         強         弱         強         弱         強
ゴルフ                                                            
しない  0.928571  0.285714 -0.714286 -0.714286 -1.071429  1.285714
する  -0.928571 -0.285714  0.714286  0.714286  1.071429 -1.285714

## 調整済み標準化誤差

In [9]:
tab = cross.values
tab

array([[2, 1, 0, 0, 0, 2],
       [1, 1, 2, 2, 3, 0]], dtype=int64)

In [10]:
col_sum = []
ind_sum = []

In [11]:
for i in range(len(tab)):
    ind_sum.append(sum(tab[i]))
ind_sum

[5, 9]

In [12]:
for i in range(len(tab[0])):
    col_sum.append(sum(tab[:,i]))
col_sum

[3, 2, 2, 2, 3, 2]

In [13]:
total = sum(col_sum)
total

14

In [14]:
fix_error = []
theory = dfe.values
actual = cross.values
for i in range(len(tab)):
    tmp = []
    for j in range(len(tab[i])):
        tmp.append((actual[i][j]-theory[i][j])/np.sqrt(theory[i][j]*(1-ind_sum[i]/total)*(1-col_sum[j]/total)))
    fix_error.append(tmp)
df_fix=pd.DataFrame(fix_error)
df_fix.columns = cross.columns
df_fix.index = cross.index
df_fix

天気          晴                 曇                  雨         
風           弱        強        弱        強         弱        強
ゴルフ                                                        
しない  1.262246  0.45542 -1.13855 -1.13855 -1.456438  2.04939
する  -1.262246 -0.45542  1.13855  1.13855  1.456438 -2.04939

## 各セルごとのp値

In [15]:
p_table = 2 * (1 - stats.norm.cdf(np.abs(fix_error)))
dfp = pd.DataFrame(p_table)
dfp.index = cross.index
dfp.columns = cross.columns
dfp

天気         晴                   曇                   雨          
風          弱         強         弱         強         弱         強
ゴルフ                                                           
しない  0.20686  0.648807  0.254891  0.254891  0.145272  0.040424
する   0.20686  0.648807  0.254891  0.254891  0.145272  0.040424

p値を確認すると天気が雨で風が強い時に有意に差が表れていることがわかる。ここで、調整済み標準化誤差を確認すると天気が雨で風が強い場合はゴルフをしない事が考えられる。